In [31]:
import os
import PIL
import cv2
import scipy
import pathlib
import numpy as np
import tensorflow as tf
import PIL.Image as Image
import tqdm as notebook_tqdm
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

## Dataset
The textile fabric database consists of 245 images of 7 different fabrics. There are 140 defect-free images, 20 for each type of fabric. With different types of defects, there are 105 images.

Images have a size of 4096×256 pixels. Defective images have been denominated as follows: nnnn_ddd_ff.png, where nnnn is the image number, ddd is the defect code, and ff is the fabric code.

In [60]:
dataset_folder = "..\\archive"  

defect_images_folder = os.path.join(dataset_folder, "Defect_images\\*")
defect_files_ds = tf.data.Dataset.list_files(defect_images_folder, shuffle = 'False')

mask_images_folder = os.path.join(dataset_folder, "Mask_images")

# flatten no defect folder  
no_defect_images_folder = os.path.join(dataset_folder, "NODefect_images\\*\\*")
no_defect_files_ds = tf.data.Dataset.list_files(no_defect_images_folder, shuffle = 'False')


print(len(defect_files_ds))
print(len(no_defect_files_ds))

# remove duplicates 
# file 0018_00_01 and _ (defect) are duplicates


106
141


In [83]:
# Defect codes
DEFECT_CODES = {
    0:  'No defect',
    2:	'Broken end',
    6:	'Broken yarn',
    10:	'Broken pick',
    16:	'Weft curling',
    19:	'Fuzzyball',
    22:	'Cut selvage',
    23:	'Crease',
    25:	'Warp ball',
    27:	'Knots',
    29:	'Contamination',
    30:	'Nep',
    36:	'Weft crack'
}

# Create encoding to keep dimensions at 13
DEFECT_ENCODINGs = {}

for i, key in zip(range(len(DEFECT_CODES)) , DEFECT_CODES.keys()):
    DEFECT_ENCODINGs[key] = i
    
@tf.function
def get_labels(file_path):
    file_name = tf.strings.split(file_path, sep='\\')[-1]
    image_code = tf.strings.split(file_name, sep='_')

    index_code = image_code[0]
    index = tf.strings.to_number(
        index_code,
        tf.int32
    )

    defect_code = image_code[1]
    defect_encode = DEFECT_ENCODINGs[defect_code.numpy()]
    defect_type = tf.strings.to_number(
        defect_encode,
        tf.int32
    )

    fabric_code = tf.strings.split(image_code[2], sep='.')[0]
    fabric_type = tf.strings.to_number(
        fabric_code,
        tf.int32
    )
    
    return index, defect_type, fabric_type

@tf.function
def process_img(file_path):
    index, defect_type, fabric_type = get_labels(file_path)
    img = tf.io.read_file(file_path)
    img = tf.io.decode_png(img, channels=1)
    
    return index, img, defect_type, fabric_type



## Dataset

index, img, defect_type, fabric_type

In [84]:
defect_ds = defect_files_ds.map(process_img)
no_defect_ds = no_defect_files_ds.map(process_img)

image_ds = defect_ds.concatenate(no_defect_ds)
len(image_ds)

AttributeError: 'SymbolicTensor' object has no attribute 'numpy'

In [80]:
for i in defect_ds:
    print(i)

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} UnknownError: {{function_node __wrapped__EagerPyFunc_Tin_1_Tout_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} UnknownError: {{function_node __wrapped__EagerPyFunc_Tin_1_Tout_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} UnboundLocalError: local variable 'defect_encode' referenced before assignment
Traceback (most recent call last):

  File "c:\Users\chris\LooprProject\Loopr_venv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 268, in __call__
    return func(device, token, args)

  File "c:\Users\chris\LooprProject\Loopr_venv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 146, in __call__
    outputs = self._call(device, args)

  File "c:\Users\chris\LooprProject\Loopr_venv\lib\site-packages\tensorflow\python\ops\script_ops.py", line 153, in _call
    ret = self._func(*args)

  File "c:\Users\chris\LooprProject\Loop

In [58]:




def decode_defect_encoding(encoding):
    pass



category_indices = list(DEFECT_ENCODINGs.keys())
unique_category_count = len(DEFECT_ENCODINGs)

def one_hot_defect_codes(index, img, defect_type, fabric_type):
    print(defect_type)
    defect_type = tf.one_hot(DEFECT_ENCODINGs[defect_type], unique_category_count)
    return index, img, defect_type, fabric_type




{0: 0, 2: 1, 6: 2, 10: 3, 16: 4, 19: 5, 22: 6, 23: 7, 25: 8, 27: 9, 29: 10, 30: 11, 36: 12}


In [59]:
tf.data.experimental.enable_debug_mode()
image_ds = image_ds.map(one_hot_defect_codes)


Tensor("args_2:0", shape=(), dtype=int32)


TypeError: in user code:

    File "C:\Users\chris\AppData\Local\Temp\ipykernel_12252\1969988553.py", line 34, in one_hot_defect_codes  *
        defect_type = tf.one_hot(DEFECT_ENCODINGs[defect_type], unique_category_count)

    TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.


In [ ]:
len(defect_files_ds)

In [ ]:
img = Image.open("..\\archive\\Defect_images\\0003_002_00.png")
img

In [ ]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)
images, labels = next(img_gen.flow_from_directory(dataset_folder))

In [ ]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

In [ ]:
image_ds = tf.data.Dataset.list_files(defect_images_folder, shuffle = 'False')
# view the files
for i in image_ds.take(5):
    print(i.numpy())

Feature Engineering

Supervised Learning

Unsupervised or Semisupervised Learning

Model Deployment/Evaluation